# Preparação do ambiente

## Bibliotecas

In [10]:
import json
import pandas as pd
import requests
import urllib.parse

from pickle import load
from tqdm.auto import tqdm

# import sys
# import codecs
# import urllib.request

In [11]:
pd.options.mode.chained_assignment = None  # default='warn'

## Constantes e funções auxiliares

In [12]:
file_model = '../models/clf_market_places_sgd.pkl'
with open(file_model, 'rb') as f:
    clf = load(f)
clf

Pipeline(steps=[('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 stop_words=['a', 'à', 'ao', 'aos', 'aquela',
                                             'aquelas', 'aquele', 'aqueles',
                                             'aquilo', 'as', 'às', 'até', 'com',
                                             'como', 'da', 'das', 'de', 'dela',
                                             'delas', 'dele', 'deles', 'depois',
                                             'do', 'dos', 'e', 'é', 'ela',
                                             'elas', 'ele', 'eles', ...])),
                ('transformer', TfidfTransformer()),
                ('clf', SGDClassifier(alpha=1e-05, loss='log_loss'))])

In [13]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'

In [14]:
def load_sch(sch_database_file):
    
    # load SCH database
    usecols = [0,1,11,12,13,14,15]
    dtype = {'Número de Homologação': 'str'}
    parse_dates = [0]
    date_format = '%d/%m/%Y'

    df_sch = pd.read_csv(
        sch_database_file,
        sep=';',
        usecols=usecols,
        dtype=dtype,
        parse_dates=parse_dates,
        date_format=date_format
        )

    df_sch = df_sch.sort_values(by='Data da Homologação',ascending=False)

    df_modelo = df_sch[['Número de Homologação','Modelo']].dropna()
    df_modelo = df_modelo.groupby('Número de Homologação',as_index=False)['Modelo'].apply(lambda x: ' | '.join(x))

    df_nome_comercial = df_sch[['Número de Homologação','Nome Comercial']].dropna()
    df_nome_comercial = df_nome_comercial.groupby('Número de Homologação',as_index=False)['Nome Comercial'].apply(lambda x: ' | '.join(x))

    columns_to_keep = ['Data da Homologação', 'Número de Homologação', 'Nome do Fabricante', 'Categoria do Produto', 'Tipo do Produto']
    df_sch = df_sch[columns_to_keep]
    df_sch = df_sch.drop_duplicates(subset='Número de Homologação')

    df_sch = df_sch.merge(df_modelo,how='left')
    df_sch = df_sch.merge(df_nome_comercial,how='left')
    df_sch = df_sch.fillna('')
    df_sch['Modelo Completo'] = df_sch[['Modelo', 'Nome Comercial']].apply(lambda row: ' | '.join(row) if len(row['Nome Comercial'])>0 else row['Modelo'],axis=1)
    
    df_sch = df_sch.reset_index(drop=True)
    
    return df_sch 

In [15]:
def parse_results(result, query=None):
    keys_to_keep = ['id', 'title', 'catalog_product_id', 'permalink', 'category_id', 'domain_id', 
                    'currency_id', 'price', 'original_price', 'sale_price', 
                    'initial_quantity', 'available_quantity', 'official_store_id', 'official_store_name']
    
    parsed_result = {}
    parsed_result['query'] = query
    for key in keys_to_keep:
        parsed_result[key] = result.get(key)
    
    if result.get('seller') is not None:
        parsed_result['seller_id'] = result.get('seller').get('id')
        parsed_result['seller_nickname'] = result.get('seller').get('nickname')
    else:
        parsed_result['seller_id'] = None
        parsed_result['seller_nickname'] = None    

    brand = None
    model = None
    gtin = None
    detailed_model = None
    anatel_homologation_number = None
    
    for attribute in result['attributes']:
        if attribute['id'] == 'BRAND':
            brand = attribute['value_name']
        elif attribute['id'] == 'MODEL':
            model = attribute['value_name']
        elif attribute['id'] == 'DETAILED_MODEL':
            detailed_model = attribute['value_name'] 
        elif attribute['id'] == 'GTIN':
            gtin = attribute['value_name'] 
        elif attribute['id'] == 'ANATEL_HOMOLOGATION_NUMBER':
            anatel_homologation_number = attribute['value_name']

    if anatel_homologation_number is None:
        item_id = parsed_result['id']
        attr_url = f'https://api.mercadolibre.com/items/{item_id}?attributes=attributes&include_internal_attributes=true'
        headers = {'user-agent': USER_AGENT}
        attr_response = requests.get(attr_url,headers=headers)
        attr_content = attr_response.content.decode(attr_response.encoding)
        attr_content =  json.loads(attr_content)
        attr_content = attr_content['attributes']
        for attr in attr_content:
            if attr['id'] == 'ANATEL_HOMOLOGATION_NUMBER':
                anatel_homologation_number = attr['value_name']

    parsed_result['brand'] = brand
    parsed_result['model'] = model
    parsed_result['detailed_model'] = detailed_model
    parsed_result['gtin'] = gtin
    parsed_result['anatel_homologation_number'] = anatel_homologation_number
    
    return parsed_result

In [16]:
def search_items(keywords):
    
    query = urllib.parse.quote_plus(' '.join(keywords))

    print('Start searching for', query)
    
    url = 'https://api.mercadolibre.com/sites/MLB/search'
    params={'q': query, 'offset': 0}
    headers = {'user-agent': USER_AGENT}
    
    results = []
    
    response = requests.get(url,params=params)
    content = response.content.decode(response.encoding)
    content =  json.loads(content)
    results.extend(content['results'])
        
    primary_results = content['paging']['primary_results']    
    # max offset shold be 1000, so we add 1 to primary_results
    next_offsets = list(range(50,primary_results+1,50))

    for offset in next_offsets:
        params={'q': query, 'offset': offset}
        response = requests.get(url,params=params)
        content = response.content.decode(response.encoding)
        content =  json.loads(content)
        results.extend(content['results'])

    print('Search complete: parsing results...')
    parsed_results = [parse_results(result,query) for result in tqdm(results)]
        
    return pd.DataFrame(parsed_results)

# Carga e prepação dos dados

In [17]:
sch_database_file = '../../certificacao-homologacao/schwebsearch/datasets/sch_database/produtos_certificados.zip'
df_sch = load_sch(sch_database_file)
df_sch.head()

,Data da Homologação,Número de Homologação,Nome do Fabricante,Categoria do Produto,Tipo do Produto,Modelo,Nome Comercial,Modelo Completo
0,2024-05-20,030572416551,Decathlon,2,Transceptor de Radiação Restrita,8605113,W900,8605113 | W900
1,2024-05-20,004732404809,Elsys Equipamentos Eletrônicos Ltda,2,Sistemas de Identificação por Radiofrequências,ESF-DE5100I | ESF-DE5100I,,ESF-DE5100I | ESF-DE5100I
2,2024-05-20,061542414894,"Shenzhen Baseus Technology Co., Ltd.",1,Acessório p/ Telefone Móvel Celular do tipo Ba...,PPAP2-10A,,PPAP2-10A
3,2024-05-20,050542403757,Lear Corporation,2,Sistemas Operando nas Faixas de RF Ultra Larga,KOBJXF23A | KOBJXF23A,,KOBJXF23A | KOBJXF23A
4,2024-05-20,062092408867,"Fortinet, Inc.",3,Equipamento de Rede de Dados,FG-120G | FG-121G,,FG-120G | FG-121G


In [18]:
file_ean_celulares = '../datasets/lista_celulares_homologados_ean.xlsx'

dtype = {'Número de Homologação': 'str', 'Código EAN': 'str'}

df_ean_celulares = pd.read_excel(file_ean_celulares,dtype=dtype)
df_ean_celulares = df_ean_celulares[df_ean_celulares['Código EAN'].str.len()>=13]

columns_to_keep = ['Código EAN', 'Número de Homologação']
df_ean_sch = df_ean_celulares[columns_to_keep].drop_duplicates()
# df_ean_sch = df_ean_sch.dropna().reset_index(drop=True)
# df_ean_sch = df_ean_sch.astype('int64')

df_ean_sch.columns = ['ean_sch', 'sch_sch']
df_ean_sch.head()

,ean_sch,sch_sch
0,7892597349623,022972000330
1,7892597349630,022972000330
2,7892597350971,013692100330
3,7892597350988,013692100330
4,7892597351749,013692100330


In [19]:
file_api_ml = '../datasets/api_ml.parquet'

# query Mercado Livre
keywords = [['celular','smartphone'],['tv','box'],['carregador','celular']]

# df_api_ml = pd.concat([search_items(keyword) for keyword in keywords])
# df_api_ml.to_parquet(file_api_ml)

# Open previous query
df_api_ml = pd.read_parquet(file_api_ml)

df_api_ml.head()

,query,id,title,catalog_product_id,permalink,category_id,domain_id,currency_id,price,original_price,...,available_quantity,official_store_id,official_store_name,seller_id,seller_nickname,brand,model,detailed_model,gtin,anatel_homologation_number
0,celular+smartphone,MLB4559118376,Samsung Galaxy A15 5g Dual Sim 128gb Azul Clar...,MLB32174378,https://www.mercadolivre.com.br/samsung-galaxy...,MLB1055,MLB-CELLPHONES,BRL,899.00,1279.0,...,500,51.0,Webfones,640436496.0,POST COMMERCE,Samsung,A15 5G Dual Sim,Light blue,7892509132794,198242300953
1,celular+smartphone,MLB3589269609,Samsung Galaxy A15 4g Dual Sim 128 Gb Azul Esc...,MLB32175033,https://www.mercadolivre.com.br/samsung-galaxy...,MLB1055,MLB-CELLPHONES,BRL,849.00,1199.0,...,5000,2962.0,Samsung,480263032.0,MERCADOLIVRE ELETRONICOS,Samsung,A15 4G,128Gb,7892509134262,198252300953
2,celular+smartphone,MLB4498736158,Motorola Moto G54 5g 256 Gb Verde 8gb Ram,MLB27912314,https://www.mercadolivre.com.br/motorola-moto-...,MLB1055,MLB-CELLPHONES,BRL,1189.00,1459.0,...,250,51.0,Webfones,640436496.0,POST COMMERCE,Motorola,G54 Dual SIM (eSIM),None,7892597353583,117032300330
3,celular+smartphone,MLB3589212353,Samsung Galaxy A15 Dual Sim 4g 256gb Azul Escu...,MLB31214368,https://www.mercadolivre.com.br/samsung-galaxy...,MLB1055,MLB-CELLPHONES,BRL,1039.00,1499.0,...,5000,2962.0,Samsung,480263032.0,MERCADOLIVRE ELETRONICOS,Samsung,A15 4G,None,7892509135368,198252300953
4,celular+smartphone,MLB3608574373,Smartphone Motorola Moto G04 128gb 8gb Ram Boo...,MLB34079279,https://www.mercadolivre.com.br/smartphone-mot...,MLB1055,MLB-CELLPHONES,BRL,660.45,849.0,...,250,1656.0,Motorola,326016487.0,MOTOROLA OFICIAL,Motorola,Moto G04,Moto G04,7892597353736,205312300330


# Análise

In [20]:
columns_to_keep = ['query', 'id', 'permalink', 'title','brand', 'model', 'gtin', 'anatel_homologation_number']
df_analise = df_api_ml[columns_to_keep]

columns_to_keep = ['query', 'id', 'permalink', 'title','brand', 'model', 'ean_mp', 'sch_mp']
df_analise.columns = columns_to_keep

# df_analise['ean_mp'] = df_analise['ean_mp'].fillna('0')
df_analise['ean_mp'] = df_analise['ean_mp'].str.split(',')
df_analise = df_analise.explode('ean_mp')
# df_analise['ean_ml'] = df_analise['ean_ml'].str.zfill(13)

df_analise['passivel'] = clf.predict(df_analise.title)
df_analise['passivel_proba'] = clf.predict_proba(df_analise.title)[:,1]

df_analise['ean_presente'] = df_analise['ean_mp'].apply(lambda x:  0 if x is None else 1)
df_analise['sch_presente'] = df_analise['sch_mp'].apply(lambda x:  0 if x is None else 1)

df_analise['sch_mp'] = df_analise['sch_mp'].fillna('0').str.zfill(12)

columns_to_merge = ['Número de Homologação', 'Tipo do Produto', 'Modelo Completo']
df_analise = df_analise.merge(df_sch[columns_to_merge], left_on='sch_mp', right_on='Número de Homologação',how='left')
df_analise['sch_valido'] = df_analise['Número de Homologação'].apply(lambda x:  0 if pd.isna(x) else 1)

# df_analise = df_analise.merge(df_ean_sch,left_on='ean_ml', right_on='ean_sch', how='left')
# df_analise['ean_ok'] = df_analise['ean_ml']==df_analise['ean_sch']
# df_analise['sch_ok'] = df_analise['sch_ml']==df_analise['sch_sch']

# columns_to_merge = ['Número de Homologação', 'Modelo Completo']
# df_analise = df_analise.merge(df_sch[columns_to_merge],left_on='sch_ml',right_on='Número de Homologação',how='left')

df_analise.head()

,query,id,permalink,title,brand,model,ean_mp,sch_mp,passivel,passivel_proba,ean_presente,sch_presente,Número de Homologação,Tipo do Produto,Modelo Completo,sch_valido
0,celular+smartphone,MLB4559118376,https://www.mercadolivre.com.br/samsung-galaxy...,Samsung Galaxy A15 5g Dual Sim 128gb Azul Clar...,Samsung,A15 5G Dual Sim,7892509132794,198242300953,1,0.995424,1,1,198242300953,Sistemas de Identificação por Radiofrequências,SM-A156M/DSN | SM-A156M/DSN | SM-A156M/DSN,1
1,celular+smartphone,MLB3589269609,https://www.mercadolivre.com.br/samsung-galaxy...,Samsung Galaxy A15 4g Dual Sim 128 Gb Azul Esc...,Samsung,A15 4G,7892509134262,198252300953,1,0.997864,1,1,198252300953,Telefone Móvel Celular,SM-A155M/DSN | SM-A155M/DSN | SM-A155M/DSN,1
2,celular+smartphone,MLB4498736158,https://www.mercadolivre.com.br/motorola-moto-...,Motorola Moto G54 5g 256 Gb Verde 8gb Ram,Motorola,G54 Dual SIM (eSIM),7892597353583,117032300330,1,0.993672,1,1,117032300330,Transceptor de Radiação Restrita,XT2343-1 | XT2343-1 | XT2343-1 | moto g54 5G |...,1
3,celular+smartphone,MLB3589212353,https://www.mercadolivre.com.br/samsung-galaxy...,Samsung Galaxy A15 Dual Sim 4g 256gb Azul Escu...,Samsung,A15 4G,7892509135368,198252300953,1,0.994379,1,1,198252300953,Telefone Móvel Celular,SM-A155M/DSN | SM-A155M/DSN | SM-A155M/DSN,1
4,celular+smartphone,MLB3608574373,https://www.mercadolivre.com.br/smartphone-mot...,Smartphone Motorola Moto G04 128gb 8gb Ram Boo...,Motorola,Moto G04,7892597353736,205312300330,1,0.996856,1,1,205312300330,Telefone Móvel Celular,XT2421-1 | XT2421-1,1


In [21]:
columns_to_group = ['query', 'id', 'passivel', 'ean_presente', 'sch_presente', 'sch_valido']
agg_func = {'id': 'count', 'passivel': 'sum', 'ean_presente': 'sum', 'sch_presente': 'sum', 'sch_valido': 'sum'}
df_resumo = df_analise[columns_to_group].groupby(columns_to_group[0]).agg(agg_func)
df_resumo['pe_ean_presente'] = df_resumo['ean_presente']/df_resumo['passivel']*100
df_resumo['pe_sch_presente'] = df_resumo['sch_presente']/df_resumo['passivel']*100
df_resumo['pe_sch_valido'] = df_resumo['sch_valido']/df_resumo['passivel']*100

columns_to_keep = ['Total de anúncios', 'Anúncios de Produtos Passíveis', 
                   'Anúncios com EAN Informado', 'Anúncios com código SCH Informado', 'Anúncios com código SCH Válido', 
                   '% Anúncios com EAN Informado', '% Anúncios com código SCH Informado', '% Anúncios com código SCH Válido']

df_resumo.columns = columns_to_keep
df_resumo.index.name = 'Palavra chave'

df_resumo

,Total de anúncios,Anúncios de Produtos Passíveis,Anúncios com EAN Informado,Anúncios com código SCH Informado,Anúncios com código SCH Válido,% Anúncios com EAN Informado,% Anúncios com código SCH Informado,% Anúncios com código SCH Válido
Palavra chave,,,,,,,,
carregador+celular,1132,1109,638,994,754,57.529306,89.630298,67.989179
celular+smartphone,1036,1035,322,991,743,31.111111,95.748792,71.787440
tv+box,1027,1004,155,879,738,15.438247,87.549801,73.505976


In [22]:
df_analise.columns

Index(['query', 'id', 'permalink', 'title', 'brand', 'model', 'ean_mp',
       'sch_mp', 'passivel', 'passivel_proba', 'ean_presente', 'sch_presente',
       'Número de Homologação', 'Tipo do Produto', 'Modelo Completo',
       'sch_valido'],
      dtype='object')

In [15]:
df_analise[(df_analise['sch_mp']!='000000000000')&(df_analise['sch_valido']==0)]

,query,id,permalink,title,brand,model,ean_mp,sch_mp,passivel,passivel_proba,ean_presente,sch_presente,Número de Homologação,Tipo do Produto,Modelo Completo,sch_valido
13,celular+smartphone,MLB4530686942,https://www.mercadolivre.com.br/motorola-moto-...,Motorola Moto G24 128gb Grafite 4gb Ram,Motorola,G24,7892597353811,020667200330,1,0.994404,1,1,NaN,NaN,NaN,0
15,celular+smartphone,MLB3497390875,https://www.mercadolivre.com.br/multilaser-e-2...,Multilaser E 2 Dual Sim 32 Gb Dourado 1 Gb Ram,Multilaser,E 2,7908414420229,001324721031,1,0.997591,1,1,NaN,NaN,NaN,0
22,celular+smartphone,MLB4799984678,https://www.mercadolivre.com.br/xiaomi-poco-c6...,Xiaomi Poco C65 Dual Sim 256 Gb Preto 8 Gb Ram,Xiaomi,Poco C65,6941812753040,012589745316,1,0.997462,1,1,NaN,NaN,NaN,0
24,celular+smartphone,MLB3698858981,https://www.mercadolivre.com.br/smartphone-lg-...,Smartphone LG K8 Plus 16gb 4g Quad-core - 1gb ...,LG,K8+ Dual SIM,7891991015462,040761660095,1,0.974970,1,1,NaN,NaN,NaN,0
39,celular+smartphone,MLB4686250494,https://produto.mercadolivre.com.br/MLB-468625...,Smartphone Realme C67 Dual Sim 256gb 8gb Nfc +...,Realme,C67,None,000028665423,1,0.997707,0,1,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,carregador+celular,MLB4721937278,https://www.mercadolivre.com.br/carregador-tur...,Carregador Turbo 20w P/ iPhone 8 Plus X Xr 11 ...,Fokus,Carregador P/ Iphone 14,07898656760446,000088111003,1,0.984525,1,1,NaN,NaN,NaN,0
3150,carregador+celular,MLB3669053453,https://produto.mercadolivre.com.br/MLB-366905...,Suporte Carregador Rápido Indução Motorola Sa...,Carregador por Indução,Indução,None,123456789123,1,0.904868,0,1,NaN,NaN,NaN,0
3169,carregador+celular,MLB3478897185,https://produto.mercadolivre.com.br/MLB-347889...,Fonte Carregador Cabo Usbc Para iPhone 15 Pro ...,Agold,Carregador Para Iphone 15 UsbC Turbo Cabo Tipo...,None,849822114100,1,0.892197,0,1,NaN,NaN,NaN,0
3178,carregador+celular,MLB4510278902,https://produto.mercadolivre.com.br/MLB-451027...,2x Fonte Turbo 3 Entradas Usb Carga Rápida 5.1...,Inova,Carregamento Rápido Android Carregar,None,000913121140,1,0.922564,0,1,NaN,NaN,NaN,0


In [18]:
df_sch[df_sch['Número de Homologação']=='175342314550']

,Data da Homologação,Número de Homologação,Nome do Fabricante,Categoria do Produto,Tipo do Produto,Modelo,Nome Comercial,Modelo Completo
462,2024-05-10,175342314550,"Xiaomi Communications Co., Ltd.",2,Transceptor de Radiação Restrita,23100RN82L | 23100RN82L | 2310FPCA4G | 23100RN...,Redmi 13C | Redmi 13C | POCO C65 | Redmi 13C |...,23100RN82L | 23100RN82L | 2310FPCA4G | 23100RN...


In [21]:
df_ean_sch[df_ean_sch['sch_sch']=='175342314550']

,ean_sch,sch_sch
415,7908426308829,175342314550
418,7908426308836,175342314550
421,7908426308843,175342314550
445,7908426308850,175342314550
456,7908426308867,175342314550
888,7908426308874,175342314550


In [23]:
df_ean_sch['ean_sch'].unique().shape

(1802,)

In [26]:
df_analise.merge(df_ean_sch,left_on='ean_mp',right_on='ean_sch',how='left')

,query,id,permalink,title,brand,model,ean_mp,sch_mp,passivel,passivel_proba,ean_presente,sch_presente,Número de Homologação,Tipo do Produto,Modelo Completo,sch_valido,ean_sch,sch_sch
0,celular+smartphone,MLB4559118376,https://www.mercadolivre.com.br/samsung-galaxy...,Samsung Galaxy A15 5g Dual Sim 128gb Azul Clar...,Samsung,A15 5G Dual Sim,7892509132794,198242300953,1,0.995424,1,1,198242300953,Sistemas de Identificação por Radiofrequências,SM-A156M/DSN | SM-A156M/DSN | SM-A156M/DSN,1,7892509132794,198242300953
1,celular+smartphone,MLB3589269609,https://www.mercadolivre.com.br/samsung-galaxy...,Samsung Galaxy A15 4g Dual Sim 128 Gb Azul Esc...,Samsung,A15 4G,7892509134262,198252300953,1,0.997864,1,1,198252300953,Telefone Móvel Celular,SM-A155M/DSN | SM-A155M/DSN | SM-A155M/DSN,1,7892509134262,198252300953
2,celular+smartphone,MLB4498736158,https://www.mercadolivre.com.br/motorola-moto-...,Motorola Moto G54 5g 256 Gb Verde 8gb Ram,Motorola,G54 Dual SIM (eSIM),7892597353583,117032300330,1,0.993672,1,1,117032300330,Transceptor de Radiação Restrita,XT2343-1 | XT2343-1 | XT2343-1 | moto g54 5G |...,1,7892597353583,117032300330
3,celular+smartphone,MLB3589212353,https://www.mercadolivre.com.br/samsung-galaxy...,Samsung Galaxy A15 Dual Sim 4g 256gb Azul Escu...,Samsung,A15 4G,7892509135368,198252300953,1,0.994379,1,1,198252300953,Telefone Móvel Celular,SM-A155M/DSN | SM-A155M/DSN | SM-A155M/DSN,1,7892509135368,198252300953
4,celular+smartphone,MLB3608574373,https://www.mercadolivre.com.br/smartphone-mot...,Smartphone Motorola Moto G04 128gb 8gb Ram Boo...,Motorola,Moto G04,7892597353736,205312300330,1,0.996856,1,1,205312300330,Telefone Móvel Celular,XT2421-1 | XT2421-1,1,7892597353736,205312300330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,carregador+celular,MLB4754504636,https://www.mercadolivre.com.br/fonte-carregad...,Fonte Carregador Para iPhone 11 Usb Tipo C 20w,T.VARIEDADES,Compatível com 11 e 12,7897651576724,107392013196,1,0.972390,1,1,107392013196,Carregador para Telefone Celular,YH-253A | YH-253A | YH-253A | YH-253A | YH-253...,1,NaN,NaN
3191,carregador+celular,MLB3317323624,https://produto.mercadolivre.com.br/MLB-331732...,Adaptador Tomada Universal Padrão Europeu E Br...,Enjoy,CP100,7898587274340,000000000000,0,0.365330,1,0,NaN,NaN,NaN,0,NaN,NaN
3192,carregador+celular,MLB3434126623,https://www.mercadolivre.com.br/carregador-dup...,Carregador Duplo Mini Turbo Rápido 20w Rock Us...,Rock,T43,None,169662213363,1,0.937064,0,1,169662213363,Carregador para Telefone Celular,CB21 | CB22,1,NaN,NaN
3193,carregador+celular,MLB3391734605,https://www.mercadolivre.com.br/tomada-usb-vei...,Tomada Usb Veicular 4.2a - 2 Saídas Substitui ...,Wr Acessórios & Cia,USB Duplo - 2.1A,7908724600199,000000000000,1,0.865808,1,0,NaN,NaN,NaN,0,NaN,NaN
